In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image

import tensorflow as tf
from tensorflow import keras

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

In [2]:
from sklearn.datasets import load_files 
from keras.utils import np_utils

from keras.preprocessing import image
from tqdm import tqdm # progress bar

data_dir = "../input/multiclass-ds6050/Multiclass"
batch_size = 10;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 224; 
img_width = 224;
n_classes = 3

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)        

dataset_size = len(train_ds)

In [ ]:
base_model = keras.applications.resnet50.ResNet50(weights='imagenet',include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation = 'tanh')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_ds,
                    steps_per_epoch=5,
                    validation_data=validation_ds,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

In [ ]:
score = model.evaluate(validation_ds)

## Sarah Code 

### Data Augmentation

In [3]:
# Create an image generator 
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1)
    ]
)

aug_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
# Create an image generator 
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
    ]
)

aug_ds2 = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
# Original Images
# Rows and columns are set to fit one training batch (32)
class_names = ['Covid-19', 'Other', 'Pneumonia']
n_rows = 2
n_cols = 4
plt.figure(figsize=(n_cols * 3, n_rows * 3))
for images, labels in train_ds.take(1):
    for i in range (n_rows*n_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis('off')
        plt.title(class_names[labels[i]], fontsize=12)
        
plt.subplots_adjust(wspace=.2, hspace=.2)

In [ ]:
# Augmented Images
# Rows and columns are set to fit one training batch (32)
class_names = ['Covid-19', 'Other', 'Pneumonia']

n_rows = 2
n_cols = 4
plt.figure(figsize=(n_cols * 3, n_rows * 3))
for images, labels in aug_ds.take(1):
    for i in range (n_rows*n_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis('off')
        print(labels[i])
        plt.title(class_names[labels[i]], fontsize=12)
        
plt.subplots_adjust(wspace=.2, hspace=.2)

### Transfer Learning

### ResNet50

### Model 1

In [ ]:
from keras.applications.resnet import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(3, activation='softmax'))

base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

resnet_history = base_model.fit(train_ds, validation_data = validation_ds, steps_per_epoch = 50, epochs = 5)

### VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

# Added this since Module 2
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

vgghist = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5,
                   callbacks=[earlystopping])

In [ ]:
score = model.evaluate(validation_ds)

### Inceptionv3

### Model 1

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.55)(x)
x = keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

# train the model on the new data for a few epochs
inceptionhist = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

In [ ]:
score = model.evaluate(validation_ds)

### Model 2

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

# train the model on the new data for a few epochs
inceptionhist = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

## DenseNet-121

### Model 1

In [6]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

### Model 2

In [ ]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

### Model 3

In [7]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 4

In [8]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

## VGG 19

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

base_model = VGG19(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

# Added this since Module 2
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

vgghist = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5,
                   callbacks=[earlystopping])

In [ ]:
score = model.evaluate(validation_ds)

## Inception ResNet V2

### Model 1

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

# train the model on the new data for a few epochs
inceptionhist = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

### Model 2

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

# train the model on the new data for a few epochs
inceptionhist = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

## DesNet201

### Model 5 - (Model 1 but with no data augmentation)

In [26]:
from keras.applications.densenet import DenseNet201

base_model = DenseNet201(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.7)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

# Using Data Augmented Training Datasets

### VGG 16

### Model 2 - (same as Model 1, but data augmentation and using the new method)

In [22]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

# Added this since Module 2
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

vgghist = model.fit(aug_ds2,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5,
                   callbacks=[earlystopping])

## VGG19

### Model 1

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

base_model = VGG19(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

# Added this since Module 2
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

vgghist = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5,
                   callbacks=[earlystopping])

### Model 2 - but new data augmentation

In [ ]:
# Create an image generator 
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
    ]
)

aug_ds2 = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [23]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

base_model = VGG19(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

# Added this since Module 2
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

vgghist = model.fit(aug_ds2,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5,
                   callbacks=[earlystopping])

## ResNet 152

In [ ]:
from keras.applications.resnet_v2 import ResNet152V2

base_model = ResNet152V2(input_shape=(224, 224,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet152V2(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(3, activation='softmax'))

base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

resnet_history = base_model.fit(aug_ds, validation_data = validation_ds, steps_per_epoch = 50, epochs = 5)

## DenseNet 201

### Model 1

In [9]:
from keras.applications.densenet import DenseNet201

base_model = DenseNet201(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.7)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 2

In [12]:
from keras.applications.densenet import DenseNet201

base_model = DenseNet201(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.8)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

### Model 3

In [13]:
from keras.applications.densenet import DenseNet201

base_model = DenseNet201(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

### Model 4 - using the new data augmentation

In [25]:
from keras.applications.densenet import DenseNet201

base_model = DenseNet201(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.7)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds2,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### DenseNet-121 (Model 4 but using data augmentation)

### Model 5

In [18]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 6 - Change the data augmentation

In [19]:
# Create an image generator 
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
    ]
)

aug_ds2 = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [20]:
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

densenet = model.fit(aug_ds2,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Xception

### Model 1

In [11]:
from keras.applications.xception import Xception

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
                                                  
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(3, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 2

In [15]:
from keras.applications.xception import Xception

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
                                                  
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(avg)
x = keras.layers.Dropout(0.7)(x)
output = keras.layers.Dense(3, activation="softmax")(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 3

In [16]:
from keras.applications.xception import Xception

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
                                                  
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(avg)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(3, activation="softmax")(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.7, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 4

In [17]:
from keras.applications.xception import Xception

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
                                                  
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(256, activation='relu')(avg)
x = keras.layers.Dropout(0.7)(x)
output = keras.layers.Dense(3, activation="softmax")(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.7, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(aug_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

### Model 3 - but new data augmentation

In [21]:
from keras.applications.xception import Xception

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
                                                  
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(avg)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(3, activation="softmax")(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.7, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(aug_ds2,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

## ResNet50

### Model 2 - same as Model 1 but the new data augmentation

In [24]:
from keras.applications.resnet import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(3, activation='softmax'))

base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

resnet_history = base_model.fit(aug_ds2, validation_data = validation_ds, steps_per_epoch = 50, epochs = 5)